In [1]:
import pandas as pd
import joblib
import numpy as np
from nltk import word_tokenize
from sklearn.metrics import (
    accuracy_score,
    confusion_matrix,
    ConfusionMatrixDisplay,
    f1_score,
    recall_score,
    precision_score,
    classification_report,
    matthews_corrcoef, 
)

In [2]:
def evaluate(y_test,y_pred):
    accuracy = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    mcc = matthews_corrcoef(y_test, y_pred)
    print("F1 Score:", f1)
    print("Recall:", recall)
    print("Precision:", precision)
    print("Accuracy:", accuracy)
    print("MCC:", mcc)
    print("\n")
    print("Confusion Matrix")
    print(pd.DataFrame(confusion_matrix(y_test, y_pred)))
    print("\n")
    print("Classification Report")
    print(classification_report(y_test, y_pred))

In [3]:
filepath_name_test = (('../../../data/mixed_dataset/test_cleaned.csv'))
df_test = pd.read_csv(filepath_name_test, encoding ='utf-8')

In [4]:
df_test = df_test[df_test['tweet_cleaned'].notna()]

# TF-IDF

In [5]:
vectorizer_tfidf = joblib.load('used_models_test_dataset/mixed_dataset/vectorizer_tfidf.joblib')

In [6]:
x_test_tfidf = vectorizer_tfidf.transform(df_test['tweet_cleaned'])

In [7]:
print(x_test_tfidf.shape)

(32466, 13198)


In [8]:
y_test = df_test['label']

In [9]:
clf_tfidf = joblib.load('used_models_test_dataset/mixed_dataset/model_nb_tfidf_comp.joblib')

In [10]:
y_pred_tfidf = clf_tfidf.predict(x_test_tfidf)

In [11]:
evaluate(y_test, y_pred_tfidf)

F1 Score: 0.5214379553326167
Recall: 0.6383974265243456
Precision: 0.44069849601292016
Accuracy: 0.7531571490174336
MCC: 0.37386257975348114


Confusion Matrix
       0     1
0  20086  5541
1   2473  4366


Classification Report
              precision    recall  f1-score   support

           0       0.89      0.78      0.83     25627
           1       0.44      0.64      0.52      6839

    accuracy                           0.75     32466
   macro avg       0.67      0.71      0.68     32466
weighted avg       0.80      0.75      0.77     32466



In [12]:
# Save Confusionmatrix
confusion_matrix_tfidf_df = pd.DataFrame(confusion_matrix(y_test, y_pred_tfidf))

In [13]:
confusion_matrix_tfidf_df

,0,1
0,20086,5541
1,2473,4366


In [14]:
confusion_matrix_tfidf_df.to_csv("evaluation_test_dataset/nb_comp_tfidf_confusion_matrix_mixed_dataset.csv", index=True)

In [15]:
# Save Classificationreport
classification_report_tfidf_df = pd.DataFrame(classification_report(y_test, y_pred_tfidf, output_dict=True)).transpose()

In [16]:
classification_report_tfidf_df

,precision,recall,f1-score,support
0,0.890376,0.783783,0.833686,25627.000000
1,0.440698,0.638397,0.521438,6839.000000
accuracy,0.753157,0.753157,0.753157,0.753157
macro avg,0.665537,0.711090,0.677562,32466.000000
weighted avg,0.795651,0.753157,0.767911,32466.000000


In [17]:
classification_report_tfidf_df.to_csv("evaluation_test_dataset/nb_mn_tfidf_classification_report_mixed_dataset.csv", index=True)

# W2V

In [18]:
vectorizer_w2v = joblib.load('used_models_test_dataset/mixed_dataset/vectorizer_w2v_param.joblib')

In [21]:
def vectorize_w2v(tweets, loaded_vectorizer, vector_size=200):
    
    x_tokenized = tweets.map(word_tokenize)    

    def w2v_vector(x_tokenized, vector_size):
            vec = np.zeros(vector_size).reshape((1, vector_size))
            count = 0
            for word in x_tokenized:
                try:
                    vec += loaded_vectorizer.wv[word].reshape((1, vector_size))
                    count += 1
                except KeyError:

                    continue
            if count != 0:
                vec /= count
            return vec
        
    tweets_w2v = np.zeros((len(x_tokenized), 200))
    for i in range(len(x_tokenized)):
        tweets_w2v[i, :] = w2v_vector(x_tokenized.iloc[i], 200)

    return tweets_w2v

In [22]:
x_test_w2v = vectorize_w2v(df_test['tweet_cleaned'], vectorizer_w2v)

In [23]:
clf_w2v = joblib.load('used_models_test_dataset/mixed_dataset/model_nb_w2v_param.joblib')

In [24]:
y_pred_w2v = clf_w2v.predict(x_test_w2v)

In [25]:
evaluate(y_test, y_pred_w2v)

F1 Score: 0.46936416184971097
Recall: 0.7123848515864892
Precision: 0.34997485812800805
Accuracy: 0.6606911846239143
MCC: 0.29602639974309153


Confusion Matrix
       0     1
0  16578  9049
1   1967  4872


Classification Report
              precision    recall  f1-score   support

           0       0.89      0.65      0.75     25627
           1       0.35      0.71      0.47      6839

    accuracy                           0.66     32466
   macro avg       0.62      0.68      0.61     32466
weighted avg       0.78      0.66      0.69     32466



In [26]:
confusion_matrix_w2v_df = pd.DataFrame(confusion_matrix(y_test, y_pred_w2v))

In [27]:
confusion_matrix_w2v_df

,0,1
0,16578,9049
1,1967,4872


In [28]:
confusion_matrix_w2v_df.to_csv("evaluation_test_dataset/nb_w2v_confusion_matrix_mixed_dataset.csv", index=True)

In [29]:
# Save Classificationreport
classification_report_w2v_df = pd.DataFrame(classification_report(y_test, y_pred_w2v, output_dict=True)).transpose()

In [30]:
classification_report_w2v_df

,precision,recall,f1-score,support
0,0.893934,0.646896,0.750611,25627.000000
1,0.349975,0.712385,0.469364,6839.000000
accuracy,0.660691,0.660691,0.660691,0.660691
macro avg,0.621954,0.679640,0.609988,32466.000000
weighted avg,0.779348,0.660691,0.691366,32466.000000


In [31]:
classification_report_w2v_df.to_csv("evaluation_test_dataset/nb_w2v_classification_report_mixed_dataset.csv", index=True)

# Glove

In [32]:
%run ../../functions/vectorize_functions.py

In [33]:
x_test_glv, x_leer, y_test_glv, y_leer, vectorizer_glv = vectorize_glove(df=df_test, test_size=1, text_column='tweet_cleaned', 
                                                                                 label_column="label")

In [34]:
print(x_test_glv.shape)

(32465, 100)


In [35]:
clf_w2v = joblib.load('used_models_test_dataset/mixed_dataset/model_nb_glv.joblib')

In [36]:
y_pred_glv = clf_w2v.predict(x_test_glv)

In [37]:
evaluate(y_test_glv, y_pred_glv)

F1 Score: 0.45923739237392375
Recall: 0.6824097090217868
Precision: 0.3460625834198428
Accuracy: 0.6614507931618666
MCC: 0.27990983430880756


Confusion Matrix
       0     1
0  16807  8819
1   2172  4667


Classification Report
              precision    recall  f1-score   support

           0       0.89      0.66      0.75     25626
           1       0.35      0.68      0.46      6839

    accuracy                           0.66     32465
   macro avg       0.62      0.67      0.61     32465
weighted avg       0.77      0.66      0.69     32465



In [38]:
confusion_matrix_glv_df = pd.DataFrame(confusion_matrix(y_test_glv, y_pred_glv))

In [39]:
confusion_matrix_glv_df

,0,1
0,16807,8819
1,2172,4667


In [40]:
confusion_matrix_glv_df.to_csv("evaluation_test_dataset/nb_glv_confusion_matrix_mixed_dataset.csv", index=True)

In [41]:
# Save Classificationreport
classification_report_glv_df = pd.DataFrame(classification_report(y_test_glv, y_pred_glv, output_dict=True)).transpose()

In [42]:
classification_report_glv_df

,precision,recall,f1-score,support
0,0.885558,0.655857,0.753593,25626.000000
1,0.346063,0.682410,0.459237,6839.000000
accuracy,0.661451,0.661451,0.661451,0.661451
macro avg,0.615810,0.669134,0.606415,32465.000000
weighted avg,0.771909,0.661451,0.691584,32465.000000


In [43]:
classification_report_glv_df.to_csv("evaluation_test_dataset/nb_w2v_classification_report_mixed_dataset.csv", index=True)